# Modelos Simples como Neuronas

La idea de este Notebook es mostrar una introducción al entrenamiento de redes neuronales usando modelos simples como la regresión lineal y logística. Veremos cómo estos modelos pueden ser pensados como "neuronas" (componentes básicos de una red neuronal densa) y cubriremos conceptos que definen cómo se entrenarán estos modelos.

> Las siguientes celdas de código deben correrse únicamente si se está trabajando desde Google Colab. Permiten clonar el repositorio desde Github a la sesión.

In [ ]:
!git clone https://github.com/JuanCruzC97/ml-stuff.git

In [ ]:
cd ml-stuff/intro-deep-learning

In [ ]:
# Librerías para el manejo de los datos.
import numpy as np
import pandas as pd

# Librerías para visualización.
import plotly.express as px
#import plotly.graph_objects as go

# Funciones propias.
from utils.datasets import make_regression_dataset

# Regresión

En esta primera parte cubrimos un problema muy simple de regresión (predicción de una variable continua). Usaremos una única variable explicativa. Comenzamos generando el dataset que utilizaremos en esta etapa. Se trata de un set de datos sencillo, con una variable explicativa `X` y una variable respuesta `y` continua.

## Dataset

Generamos y exploramos un poco el dataset de regresión.

In [2]:
# Generamos los datasets de entrenamiento y evaluación.
train = make_regression_dataset(n_samples=100, noise=1.25, random_state=42)
test = make_regression_dataset(n_samples=30, noise=1.25, random_state=65)

In [3]:
# Vemos algunos valores de la variable explicativa y la respuesta.
train.head()

,X,y
0,-1.003679,-2.201476
1,3.605714,5.682870
2,1.855952,5.638039
3,0.789268,0.695674
4,-2.751851,-1.445551


In [4]:
# También podemos ver algunos estadísticos de ambas variables.
train.describe()

,X,y
count,100.000000,100.000000
mean,-0.238554,1.697763
std,2.379915,3.485939
min,-3.955823,-4.706355
25%,-2.454394,-1.157034
50%,-0.286860,1.241799
75%,1.841625,4.993439
max,3.895095,8.287708


In [5]:
px.scatter(data_frame=train,
           x="X",
           y="y",
           color_discrete_sequence=["#3d5a80"],
           height=500,
           width=800,
           template="plotly_white")

## Modelo Lineal

Por la forma de los datos visualizados podemos darnos cuenta que un modelo lineal simple con la variable `x` tendrá un ajuste pobre, vemos que hay una relación entre la variable `y` y la variable `x` pero esta relación es no es lineal.

$$ y = f(x) + \tilde{\epsilon} \qquad \text{ donde f() es no lineal} $$

El modelo de regresión lineal simple sigue la siguiente expresión, donde $w_0$ y $w_1$ son los parámetros del modelo. Los valores óptimos de estos parámetros debemos encontrarlos, de manera que lleguemos a los valores para los cuales el modelo realice la mejor predicción de la respuesta `y` en función de `x`.

$$\hat{y} = w_0 + w_1 * x$$

¿Cómo conseguimos los mejores valores para los parámetros del modelo? Para eso necesitamos una forma de medir la diferencia entre el valor observado de $y$ para cada $x$ y el valor de predicción del modelo $\hat{y}$ para los $x$ correspondientes. La métrica que usamos para medir esta diferencia es conocida como Suma de Errores Cuadráticos, básicamente, la sumatoria de los residuos elevados al cuadrado para todas las observaciones que tenemos.

$$S = \sum{(y-\hat{y})^2} \implies S = \sum{(y- w_0 + w_1*x)^2} $$

Vemos que la función que mide cuán bueno es el modelo en la predicción tiene como componentes a $y$ y $x$ que son los datos conocidos (observaciones) y también a los parámetros del modelo, por lo que la nuestra función de error depende de los parámetros del modelo $S = f(w_0, w_1)$, por lo que encontrar los valores de los parámetros (también llamados *weights*) óptimos para predecir $\hat{y}$ es un problema de optimización donde buscamos encontrar los valores de $w_0$ y $w_1$ que minimicen $S$.

$${\displaystyle \min _{w_0,w_1}\;S} = \sum{(y- w_0 + w_1*x)^2}$$

Gracias a Carl Friedrich Gauss tenemos el método de los cuadrados mínimos, con el que obtenemos los valores óptimos de los coeficientes $w_0$ y $w_1$ que se ajustan a los datos dados.

In [6]:
# Importamos la función del modelo lineal.
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [7]:
# Iniciamos el modelo.
linear_model_sk = LinearRegression()

# Entrenamos el modelo con los datos de train.
linear_model_sk.fit(X=train[["X"]],
                    y=train["y"])

LinearRegression()

Obtenemos el valor óptimo de los parámetros del modelo lineal simple para 

In [8]:
print(f'Pendiente: {round(linear_model_sk.coef_[0], 2)}')
print(f'Intercepto: {round(linear_model_sk.intercept_, 2)}')

Pendiente: 1.03
Intercepto: 1.94


In [9]:
train["y_preds1"] = linear_model_sk.predict(train[["X"]])
test["y_preds1"] = linear_model_sk.predict(test[["X"]])

print(f'Error Absoluto Promedio {round(mean_absolute_error(train["y"], train["y_preds1"]), 2)}')
print(f'Error Absoluto Promedio {round(mean_absolute_error(test["y"], test["y_preds1"]), 2)}')

Error Absoluto Promedio 2.02
Error Absoluto Promedio 1.84


In [10]:
plot = px.scatter(data_frame=train,
                  x = "X",
                  y = ["y", "y_preds1"],
                  color_discrete_sequence=["#3d5a80", "#ff6700"],
                  height=500,
                  width=800,
                  template ="plotly_white")

plot.show()

## Regresión Lineal como Neurona

En esta sección la idea es armar el mismo modelo que armamos previamente (regresión lineal simple) pero usando la librería `Keras` (desarrollada por Google para programar redes neuronales). Podemos pensar que este es el modelo más simple que podemos armar con esta librería, una regresión lineal sería una red neuronal de una única neurona (no tiene mucho de red).

El modelo recibe el input `x` (incluimos otro input igual a uno que irá multiplicado por el intercepto $w_0$). Dentro de nuestra unidad o neurona se lleva a cabo la operación $F(x)$ que da como resultado la predicción del modelo $\hat{y}$. Multiplicamos cada input por su parámetro y sumamos toda la expresión.

$$F(x) = w_1*x + w_0*1 = w_1*x + w_0 = \hat{y}$$

![Modelo Lineal](assets/linear.jpg)

Esta es una manera diferente de pensar el mismo modelo que habíamos armado y que puede ayudar un poco para entender cómo se formarán las redes neuronales más complejas.

Además de crear el mismo modelo que creamos previamente vamos a entrenarlo, es decir, obtener los parámetros óptimos $w_0$ y $w_1$ que minimizan la diferencia entre la respuesta observada y la predicha. Este proceso lo realizamos usando el método de los cuadrados mínimos, pero ahora vamos a hacerlo de manera diferente. Vamos a entrenar este modelo (aprender los parámetros óptimos) usando el método del descenso del gradiente.


In [11]:
import tensorflow as tf
from tensorflow import keras

from utils.datasets import get_fit_data, get_training_preds

In [12]:
# Hiperparámetros del entrenamiento.
LOSS = "mean_squared_error"
BATCH_SIZE = 20
EPOCHS = 25
LR = 0.01

Podemos ver que en este método de construcción del modelo tenemos muchos más parámetros a definir y debemos realizar una mayor cantidad de pasos. Para una regresión lineal simple es excesivo pero estos pasos generalizan para la construcción de cualquier tipo de red neuronal usando Keras.

Pasos a realizar:

0. Iniciamos los parámetros de la red de manera aleatoria usando una distribución normal. Podemos no incluir este paso en el código y se realizará de manera automática, pero de esta forma nos aseguramos de que sea reproducible.
   
1. Definimos las capas que formarán el modelo, esto define la arquitectura de la red neuronal. Para una regresión simple vamos a tener una capa de input (solo incluimos el shape de los datos que ingresan al modelo) y una capa de output que tiene una única neurona.
   
2. Creamos el modelo pasando las capas de input y output.
   
3. Compilamos el modelo definiendo la función a optimizar *Loss* y el método de optimización usado *SGD - Stochastic Gradient Descent*.
   
4. Finalmente entrenamos al modelo pasando los datos de *train* `X` e `y`. En este paso también definimos hiperparámetros del entrenamiento como el *batch size* y el número de *epochs*.

In [25]:
# Paso 0: Iniciamos los parámetros aleatorios.
keras.backend.clear_session()
tf.random.set_seed(123)
initial_weights = keras.initializers.RandomNormal(0, 0.05, 123)

# Paso 1: Definimos la arquitectura de la red neuronal.
# Para este caso definimos dos capas, una capa de input y una capa de output.
inputs = keras.Input(shape=(1,), name="input")
outputs = keras.layers.Dense(1, kernel_initializer=initial_weights, name="output")(inputs)

# Paso 2: Construimos el modelo uniendo capas.
linear_model_keras = keras.Model(inputs, outputs)

# Paso 3: Compilamos el modelo eligiendo la función a optimizar y el método de optimización.
linear_model_keras.compile(optimizer=keras.optimizers.SGD(learning_rate=LR), loss=LOSS)

# Paso 4: Entrenamos el modelo.
fit_history = linear_model_keras.fit(x=train[["X"]], 
                                     y=train["y"], 
                                     batch_size=BATCH_SIZE, 
                                     epochs=EPOCHS, 
                                     shuffle=True)

Epoch 1/25
5/5 [==============================] - 1s 4ms/step - loss: 12.6848
Epoch 2/25
5/5 [==============================] - 0s 5ms/step - loss: 10.0842
Epoch 3/25
5/5 [==============================] - 0s 5ms/step - loss: 8.8475
Epoch 4/25
5/5 [==============================] - 0s 5ms/step - loss: 8.2094
Epoch 5/25
5/5 [==============================] - 0s 5ms/step - loss: 7.8394
Epoch 6/25
5/5 [==============================] - 0s 3ms/step - loss: 7.5036
Epoch 7/25
5/5 [==============================] - 0s 5ms/step - loss: 7.2324
Epoch 8/25
5/5 [==============================] - 0s 4ms/step - loss: 7.0676
Epoch 9/25
5/5 [==============================] - 0s 6ms/step - loss: 6.8872
Epoch 10/25
5/5 [==============================] - 0s 6ms/step - loss: 6.8655
Epoch 11/25
5/5 [==============================] - 0s 6ms/step - loss: 6.6941
Epoch 12/25
5/5 [==============================] - 0s 6ms/step - loss: 6.6000
Epoch 13/25
5/5 [==============================] - 0s 5ms/step - loss: 

Podemos ver un resumen del modelo, donde muestra que tenemos 2 "capas" ,siendo una de inputs sin parámetros que afecten a los datos y otra de output donde tenemos 2 parámetros (la pendiente $w_1$ y el intercepto del modelo $w_0$).

In [26]:
linear_model_keras.summary()
keras.utils.plot_model(linear_model_keras)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 1)]               0         
                                                                 
 output (Dense)              (None, 1)                 2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [106]:
# 
print(f"Parámetros del modelo Lineal {[linear_model_sk.coef_[0], linear_model_sk.intercept_]}")

Parámetros del modelo Lineal [1.026409914187006, 1.9426174913286312]


In [33]:
# Podemos ver los valores que toman los parámetros del modelo con este método de optimización.
linear_model_keras.get_weights()

[array([[1.0167941]], dtype=float32), array([1.7801245], dtype=float32)]

In [34]:
train["y_preds2"] = linear_model_keras.predict(train[["X"]], verbose=0)
test["y_preds2"] = linear_model_keras.predict(test[["X"]], verbose=0)

print(f'Error Absoluto Promedio {round(mean_absolute_error(train["y"], train["y_preds2"]), 2)}')
print(f'Error Absoluto Promedio {round(mean_absolute_error(test["y"], test["y_preds2"]), 2)}')

Error Absoluto Promedio 2.05
Error Absoluto Promedio 1.82


Podemos ver cómo fue "aprendiendo" el modelo durante el entrenamiento. El siguiente gráfico muestra el valor de la función a optimizar (*loss*) para cada iteración por el dataset completo (*epoch*). Ahora veremos en mayor detalle cómo se realiza este entrenamiento.

In [142]:
fit_data = get_fit_data(fit_history, 'mse')

px.line(fit_data, 
        x=fit_data.index, 
        y="mse",
        color_discrete_sequence=["#3d5a80"],
        height=500,
        width=800,
        template="plotly_white")

### Entrenamiento

Podemos modificar los hiperparámetros del entrenamiento de la red para ver cómo los mismos afectan la dinámica del aprendizaje.

Interesante, probar:
* loss = "mean_absolute_error"
* batch size = 1
* batch size = 100
* learning rate = 0.1
* learning rate = 0.001


In [126]:
LOSS = "mean_squared_error"
BATCH_SIZE = 20
EPOCHS = 30
LR = 0.01

tf.random.set_seed(123)
initial_weights = keras.initializers.RandomNormal(0, 0.25, 123)

inputs = keras.Input(shape=(1,))
outputs = keras.layers.Dense(1, kernel_initializer=initial_weights)(inputs)

model1 = keras.Model(inputs, outputs)

model1.compile(optimizer=keras.optimizers.SGD(learning_rate=LR), loss=LOSS)

df_weights, df_preds = get_training_preds(train, model1, BATCH_SIZE, EPOCHS)

In [127]:
px.scatter(data_frame=df_preds,
           x="X",
           y=["y", "y_pred"],
           animation_frame="epoch",
           color_discrete_sequence=["#3d5a80", "#ff6700"],
           height=500,
           width=800,
           template="plotly_white").show()

px.line(df_weights, 
        x=df_weights.index, 
        y="loss",
        color_discrete_sequence=["#3d5a80"],
        height=500,
        width=800,
        template="plotly_white")

## Agregando Capas Intermedidas

Agregando capas densas intermedias a la red le damos mayor capacidad de ajuste o mayor expresividad para representar los datos. Esto permite lograr mejores ajustes.

En primer lugar podemos ver qué sucede cuando agregamos más parámetros (agregando la capa intermedia) pero con activaciones lineales. Veremos que no hay diferencia alguna que la regresión lineal anterior.

Cuando agregamos la capa intermedia con activaciones no lineales el modelo logra ajustarse muy bien a los datos.

In [35]:
LOSS = "mse"
BATCH_SIZE = 20
EPOCHS = 25
LR = 0.1

tf.random.set_seed(123)
initial_weights = keras.initializers.RandomNormal(0, 0.25, 123)

inputs = keras.Input(shape=(1,))
hidden = keras.layers.Dense(8, activation="sigmoid")(inputs)
outputs = keras.layers.Dense(1)(hidden)

model2 = keras.Model(inputs, outputs)

model2.compile(optimizer=keras.optimizers.Adam(learning_rate=LR), loss=LOSS)

df_weights, df_preds = get_training_preds(train, model2, BATCH_SIZE, EPOCHS)

px.scatter(data_frame=df_preds,
           x="X",
           y=["y", "y_pred"],
           animation_frame="epoch",
           color_discrete_sequence=["#3d5a80", "#ff6700"],
           height=500,
           width=800,
           template="plotly_white").show()

px.line(df_weights, 
        x=df_weights.index, 
        y="loss",
        color_discrete_sequence=["#3d5a80"],
        height=500,
        width=800,
        template="plotly_white").show()

In [ ]:
train["y_preds3"] = model2.predict(train[["X"]], verbose=0)
test["y_preds3"] = model2.predict(test[["X"]], verbose=0)

print(f'Error Absoluto Promedio {round(mean_absolute_error(train["y"], train["y_preds3"]), 2)}')
print(f'Error Absoluto Promedio {round(mean_absolute_error(test["y"], test["y_preds3"]), 2)}')

## Featuring Engineering

Claramente este problema muy simple no es recomendable encararlo usando la red neuronal desarrollada anteriormente. El problema permitió ver la flexibilidad de este modelo para ajustarse a los datos cuando le damos mayor flexibilidad con más neuronas en la capa intermedia y activaciones no lineales.

Podemos resolver el problema de manera simple con la regresión lineal original podemos transformar los datos de entrada viendo los residuos de los modelos anteriores.

In [1]:
px.scatter(data_frame=train,
           x = "X",
           y = train["y"] - train["y_preds1"],
           color_discrete_sequence=["#3d5a80"],
           height=500,
           width=800,
           template ="plotly_white").show()

NameError: name 'train' is not defined

In [2]:
# Agregamos la transformación senoidal de la variable.
train["sin_X"] = np.sin(train["X"])

# Entrenamos con las nuevas variables.
linear_model_t = LinearRegression()
linear_model_t.fit(X=train[["X", "sin_X"]],
                   y=train["y"])

# Guardamos la predicción del nuevo modelo.
train["y_preds4"] = linear_model_t.predict(train[["X", "sin_X"]])

NameError: name 'np' is not defined

In [ ]:
px.scatter(data_frame=train,
           x = "X",
           y = train["y"] - train["y_preds4"],
           color_discrete_sequence=["#3d5a80"],
           height=500,
           width=800,
           template ="plotly_white")

In [42]:
# Creamos la transformación cuadrática de la variable.
train["squared_X"] = train["X"]**2

# Reentrenamos con las nuevas transformaciones.
linear_model_t.fit(X=train[["X", "sin_X", "squared_X"]],
                    y=train["y"])

# Guardamos la predicción.
train["y_preds5"] = linear_model_t.predict(train[["X", "sin_X", "squared_X"]])

In [43]:
px.scatter(data_frame=train,                  
           x = "X",
           y = train["y"] - train["y_preds5"],
           color_discrete_sequence=["#3d5a80"],
           height=500,
           width=800,
           template ="plotly_white")

In [ ]:
train["y_preds6"] = linear_model_t.predict(train[["X", "sin_X", "squared_X"]])
#test["y_preds6"] = linear_model_t.predict(test[["X", "sin_X", "squared_X"]])

print(f'Error Absoluto Promedio {round(mean_absolute_error(train["y"], train["y_preds6"]), 2)}')
#print(f'Error Absoluto Promedio {round(mean_absolute_error(test["y"], test["y_preds6"]), 2)}')

# Clasificación

proximamente...